by險別，每年 *新,續,回,流失* 人數
*續* 用2021的續保單號核對

In [1]:
import os
import pandas as pd
import numpy as np
import tqdm, datetime
import pickle


from dataclasses import dataclass, field

os.getcwd()
ROUTES = ["JB"]#['KA', 'CA', 'JB', 'BA', 'KB']

def nodes_new(new):
    if new.empty:
        return pd.DataFrame()
    data_new = []
    for dst in ROUTES+["others"]:
        mask = new.str.match(f'^(?=.*{dst}).*')
        data_new.append(
            {"src_year": y, "src":"new", "dst_year": None, "dst": dst, "value": sum(mask), "tag":"新客"}
        )
    return pd.DataFrame(data_new)

def nodes_loss(loss):
    if loss.empty:
        return pd.DataFrame()
    data_loss = []
    for src in ROUTES+["others"]:
        mask = loss.str.match(f'^(?=.*{src}).*')
        data_loss.append(
            {"src_year": y, "src":src, "dst_year": None, "dst": "loss", "value": sum(mask), "tag":"流失"}
        )
    return pd.DataFrame(data_loss)

def nodes_trans(trans):
    if trans.empty:
        return pd.DataFrame()
    data_trans = []
    for dst in ROUTES+["others"]:
        for src in ROUTES+["others"]:
            mask_dst = trans["dst"].str.match(f'^(?=.*{dst}).*')
            mask_src = trans["src"].str.match(f'^(?=.*{src}).*')
            t = trans[mask_dst & mask_src]
            data_trans.append(
                {"src_year": y-1, "src":src, "dst_year": y, "dst": dst, "value": len(t), "tag":"續保"}
            )
    return pd.DataFrame(data_trans)
        
def nodes_longTimeAgo(long_time_ago):
    if long_time_ago.empty:
        return pd.DataFrame()
    data_longTimeAgo = []
    for dst in ROUTES+["others"]:
        for src in ROUTES+["others"]:
            mask_dst = long_time_ago["dst"].str.match(f'^(?=.*{dst}).*')
            mask_src = long_time_ago["src"].str.match(f'^(?=.*{src}).*')
            t = long_time_ago[mask_dst & mask_src]
            for y_t, cnt in t["year"].value_counts().iteritems():
                data_longTimeAgo.append(
                    {"src_year": y_t, "src":src, "dst_year": y, "dst": dst, "value": cnt, "tag":"回頭"}
                )
    return pd.DataFrame(data_longTimeAgo)


In [2]:
raw_data = pd.concat(
    [
        pd.read_csv(r'D:\新安\data\訂單整合資料\car_policy_full.txt', sep='|', dtype=str),
        pd.read_csv(r'D:\新安\data\訂單整合資料\fir_policy_full.txt', sep='|', dtype=str),
        pd.read_csv(r'D:\新安\data\訂單整合資料\hth_policy_full.txt', sep='|', dtype=str)
    ]
)

# columns typing:
raw_data[['birthY', 'year', 'plyAmt', 'clmCnt', 'clmAmt', 'clmRec', 'tg_year']] = raw_data[[
    'birthY', 'year', 'plyAmt', 'clmCnt', 'clmAmt', 'clmRec', 'tg_year']].astype(float)
raw_data["date"] = pd.to_datetime(raw_data["date"])
# new columns:
raw_data["cur_age"] = raw_data["year"] - raw_data["birthY"]
raw_data["cur_car_age"] = raw_data["year"] - raw_data["tg_year"]
raw_data["iply_person"] = raw_data["ipolicy"] + raw_data["id"]

In [3]:
used_data = raw_data.query("""
    date >= '2017-01-01' & \
    plyAmt >= 0 & \
    date <= '2022-03-31' & \
    cate != 'H_CV'
    """).copy()

非探討通路歸類於"others"

In [4]:
used_data["iroute"] = used_data["iroute"].apply(lambda x: x if x in ROUTES else "others")
used_data.loc[used_data["cate"].str.startswith("C"), "cate"] = "Car"

In [5]:
from dataclasses import field

@dataclass
class personIroute_year:
    cate:str = ''
    data:pd.DataFrame = field(default_factory=pd.DataFrame)

@dataclass
class iroute_record:
    car:personIroute_year = personIroute_year(cate="車險")
    fir:personIroute_year = personIroute_year(cate="住火")
    h_p:personIroute_year = personIroute_year(cate="個傷")
    h_t:personIroute_year = personIroute_year(cate="旅平")
    h_g:personIroute_year = personIroute_year(cate="團傷")

    def update(self, suborder:pd.DataFrame, cate:str):
        data = suborder.pivot_table(
            index="id",
            columns="year",
            values="iroute",
            aggfunc=lambda x: ','.join(x.unique())
        )
        if cate == "car":
            self.car.data = data
        if cate == "fir":
            self.fir.data = data
        if cate == "h_p":
            self.h_p.data = data
        if cate == "h_t":
            self.h_t.data = data
        if cate == "h_g":
            self.h_g.data = data



In [6]:
IR_cate = iroute_record()
print("car")
s = datetime.datetime.now()
IR_cate.update(used_data.query(" cate=='Car' "), cate="car")
print(f"elasped: {datetime.datetime.now()-s}")

print("fir")
s = datetime.datetime.now()
IR_cate.update(used_data.query(" cate=='F_PS' "), cate="fir")
print(f"elasped: {datetime.datetime.now()-s}")

print("H_PS")
s = datetime.datetime.now()
IR_cate.update(used_data.query(" cate=='H_PS' "), cate="h_p")
print(f"elasped: {datetime.datetime.now()-s}")

print("H_TV")
s = datetime.datetime.now()
IR_cate.update(used_data.query(" cate=='H_TV' "), cate="h_t")
print(f"elasped: {datetime.datetime.now()-s}")

print("H_GP")
s = datetime.datetime.now()
IR_cate.update(used_data.query(" cate=='H_GP' "), cate="h_g")
print(f"elasped: {datetime.datetime.now()-s}")

car
elasped: 0:04:26.387568
fir
elasped: 0:00:27.533158
H_PS
elasped: 0:00:47.947664
H_TV
elasped: 0:00:28.826066
H_GP
elasped: 0:00:41.168874


In [7]:
with open("IR_cate.pickle", "wb") as f:
    pickle.dump(IR_cate, f)

In [8]:
with open("IR_cate.pickle", "rb") as f:
    IR_cate = pickle.load(f)

In [9]:

def get_last_route(df):
    ans = pd.DataFrame(columns=["year", "iroute"], index=df.index)
    for c in df.columns:
        mask_c = ~df[c].isna()
        ans.loc[mask_c, "year"] = c
        ans.loc[mask_c, "iroute"] = df.loc[mask_c, c]
    return ans

In [ ]:
pt
nodes_new(pt)

In [10]:
pt = IR_cate.car.data

start_year = pt.columns.astype(int).min()
end_year = pt.columns.astype(int).max()

data = []
for y in range(2017, 2023):
    
    if y == start_year:
        mask = {
            "new": np.ones(len(pt), dtype=bool),
            "trans": np.zeros(len(pt), dtype=bool),
            "prev_old": np.zeros(len(pt), dtype=bool),
        }
        new = pt.loc[mask["new"], y].dropna()
        trans = pd.DataFrame()
        long_time_ago = pd.DataFrame()
    else:
        mask = {
            "new": (~pt[y].isna()) & (pt[np.arange(start_year, y)].isna().all(axis=1)),
            "trans": (~pt[y].isna()) & (~pt[y-1].isna()),
            "prev_old": (~pt[y].isna()) & (pt[y-1].isna()) & ((~pt[np.arange(start_year, y-1)].isna()).any(axis=1))
        }
        new = pt.loc[mask["new"], y]
        trans = pt.loc[mask["trans"], [y-1, y]].rename(columns={y-1:"src", y:"dst"})
        long_time_ago = get_last_route(
                pt.loc[mask["prev_old"], range(start_year, y-1)]
            ).rename(
                columns={"iroute": "src"}
            ).assign(
                dst = pt.loc[mask["prev_old"], y]
            )

    loss = pt.loc[(~pt[y].isna()) & (pt[y+1].isna()), y] if y!=end_year else pt[y].dropna()
   
    print(y, end='')
    print(" 新: ", len(new), end='')
    print(" 續: ", len(trans), end='')
    print(" 回: ", len(long_time_ago), end='')
    print(" 流失(發生在明年): ", len(loss), end='')
    print()

    res = pd.concat(
        [
            nodes_new(new),
            nodes_trans(trans),
            nodes_longTimeAgo(long_time_ago),
            nodes_loss(loss)
        ]
    )
    data.append(res)


2017 新:  789053 續:  0 回:  0 流失(發生在明年):  235753
2018 新:  224514 續:  553300 回:  0 流失(發生在明年):  216601
2019 新:  205438 續:  561213 回:  50826 流失(發生在明年):  218568
2020 新:  222176 續:  598909 回:  57659 流失(發生在明年):  230420
2021 新:  198136 續:  648324 回:  67135 流失(發生在明年):  709923
2022 新:  45137 續:  203672 回:  23395 流失(發生在明年):  272204


In [78]:
data_con = pd.concat(data)
data_con.query("dst=='others'")

,src_year,src,dst_year,dst,value
5,2017,new,None,others,86993
5,2018,new,None,others,41177
30,2017,KA,2018,others,5279
31,2017,CA,2018,others,2526
32,2017,JB,2018,others,769
...,...,...,...,...,...
139,2017,KB,2022,others,25
140,2020,others,2022,others,1471
141,2019,others,2022,others,200
142,2018,others,2022,others,45


In [11]:
data_con = pd.concat(data)
data_con.to_excel("逐年來源分析_詳細版_20220906.xlsx")

In [302]:
data_con.query(" src_year==2017 & src=='new' ")

,src_year,src,dst_year,dst,value
0,2017,new,None,KA,NaN
1,2017,new,None,CA,NaN
2,2017,new,None,JB,NaN
3,2017,new,None,BA,NaN
4,2017,new,None,KB,NaN


In [299]:
data_con

,src_year,src,dst_year,dst,value
0,2017,new,None,KA,NaN
1,2017,new,None,CA,NaN
2,2017,new,None,JB,NaN
3,2017,new,None,BA,NaN
4,2017,new,None,KB,NaN
...,...,...,...,...,...
0,2022,KA,None,loss,57337.0
1,2022,CA,None,loss,50514.0
2,2022,JB,None,loss,74495.0
3,2022,BA,None,loss,41063.0


In [238]:


def nodes_new(new):
    data_new = []
    for dst in ROUTES:
        mask = new.str.match(f'^(?=.*{dst}).*')
        data_new.append(
            {"src_year": y, "src":"new", "dst_year": None, "dst": dst, "value": sum(mask)}
        )
    return pd.DataFrame(data_new)

def nodes_loss(loss)
    data_loss = []
    for src in ROUTES:
        mask = loss.str.match(f'^(?=.*{src}).*')
        data_loss.append(
            {"src_year": y, "src":src, "dst_year": None, "dst": "loss", "value": sum(mask)}
        )
    return pd.DataFrame(data_loss)

def nodes_trans(trans):
    data_trans = []
    for dst in ROUTES:
        for src in ROUTES:
            mask_dst = trans["dst"].str.match(f'^(?=.*{dst}).*')
            mask_src = trans["src"].str.match(f'^(?=.*{src}).*')
            t = trans[mask_dst & mask_src]
            data_trans.append(
                {"src_year": y-1, "src":src, "dst_year": y, "dst": dst, "value": len(t)}
            )
    return pd.DataFrame(data_trans)
        
def nodes_longTimeAgo(long_time_ago):
    data_longTimeAgo = []
    for dst in ROUTES:
        for src in ROUTES:
            mask_dst = long_time_ago["dst"].str.match(f'^(?=.*{dst}).*')
            mask_src = long_time_ago["src"].str.match(f'^(?=.*{src}).*')
            print(src, dst)
            t = long_time_ago[mask_dst & mask_src]
            for y_t, cnt in t["year"].value_counts().iteritems():
                data_longTimeAgo.append(
                    {"src_year": y_t, "src":src, "dst_year": y, "dst": dst, "value": cnt}
                )
    return pd.DataFrame(data_longTimeAgo)



JB       11442
BA        7906
JF        7049
CA        5809
KB        5184
         ...  
JZ,CA        1
KA,I9        1
PA           1
BA,JB        1
CA,PC        1
Name: 2022.0, Length: 83, dtype: int64

In [207]:
pd.DataFrame(data_trans)["value"].sum() / len(trans)

0.9831690168506225

In [236]:
long_time_ago.query(" src.str.match(r'^(?=.*KA).*') & dst.str.match(r'^(?=.*KA).*') & year==2017")

,year,src,dst
id,,,
03fca2039c32a2fdd17bb9e561e302d5abf278f4,2017,KA,KA
0407bb551a37b8f389ed5ae7d3ab934a547aade6,2017,KA,KA
064f0c76f3f8f8138517b33518757d3aedc554b4,2017,KA,KA
070680b5620936bba0d6eac824b2ff91e74d6e4e,2017,KA,KA
07b25edac6c80c216f0ca86bfdd591c425edb9f1,2017,KA,KA
...,...,...,...
faf27f784658bf073979a6d47ff59ce9a8c1c8e6,2017,KA,KA
fb95b24a56085f08b2cc822101b569fd0add68e9,2017,KA,KA
fc9b9acd2d1260cc74862aa43130e1c74b7e318a,2017,KA,KA


In [233]:
pd.DataFrame(data_longTimeAgo).query(" src=='KA' & dst=='KA' ")
# .groupby("dst_year")["value"].sum()

,src_year,src,dst_year,dst,value
0,2020,KA,2022,KA,3243
1,2019,KA,2022,KA,306
2,2018,KA,2022,KA,91
3,2017,KA,2022,KA,86


In [217]:
mask_dst = long_time_ago["dst"].str.match(f'^(?=.*{dst}).*')
mask_src = long_time_ago["src"].str.match(f'^(?=.*{src}).*')

long_time_ago[mask_dst & mask_src]

,year,src,dst
id,,,
0045b6ff6d011347fd2b6edd3e2cd736b90c9b94,2020,KA,KA
004a035b7a176e2024ca7145765269462be08418,2020,KA,KA
004a11ac70c38355329b002b32f48bc99afb6ebc,2020,KA,KA
006461429fa73b8762f550b4ff8f211791e30221,2020,KA,KA
0064a4a5ce9fa3bbc7ca2a3ce342e65e8e789c48,2020,KA,KA
...,...,...,...
ffc98f21d7a3a6cd88042aea7320efa519c8f208,2017,KA,KA
ffd66e4692214877443b970d1ce5877e463f7c9b,2020,KA,KA
ffdeaa13a84ecc72bf8e383cf5edda08bface346,2020,"KA,PC,BA",KA


e.g. y=2020

新:
new = ~t[y].isna() & t[[i for i in columns if y<2020]].isna().all(axis=1)

續:
trans = ~t[y].isna() & ~t[y-1].isna()

回頭: 比較複雜，因為每個人都要用最後一年來判斷
old_p1 = t[[i for i in columns if y<2020]].apply(lambda x: x.iloc[-1], axis=0)
old = ~t[y].isna() & t[y-1].isna() & ~old_p1.isna()

In [32]:
# used_data_ = used_data.copy()
# used_data_.loc[used_data_["cate"].str.startswith("C"), "cate"] = "Car"
# used_data_.groupby(["cate", "year"])["id"].nunique()

for y in range(2017, 2021):
    prev = used_data.query(" cate=='Car' & year==@y-1 ")
    this = used_data.query(" cate=='Car' & year==@y ")
    
    print(y)
    print('續:', this.query(" id.isin(@prev['id']) ")["id"].nunique())
    print('新:', this.query(" id.isin(@prev['id']) ")["id"].nunique())

0
553300
561213
598909


In [8]:
used_data.loc[used_data["cate"].str.startswith("C"), "cate"] = "Car"

In [ ]:

used_data.loc[used_data["cate"].str.startswith("C"), "cate"] = "Car"

data_holder = []
for i in tqdm.tqdm(["Car"]):
    print(f"cate={i}")
    for y in range(2017, 2022):
        print(f"year={y}")
        s = datetime.datetime.today()
        thisYear = used_data.query("""
                    year==@y & \
                    cate==@i
                    """)["id"].astype(str)
        prevYear = used_data.query("""
                        year==@y-1 & \
                        cate==@i
                        """)["id"].astype(str)
        long_time_ago = used_data.query("""
                    year<@y-1 & \
                    cate==@i
                    """)["id"].astype(str)
        new = thisYear[~thisYear.isin(prevYear) & ~thisYear.isin(long_time_ago)].nunique()
        trans = thisYear[thisYear.isin(prevYear)].nunique() #  & ~thisYear.isin(long_time_ago)
        prev_old = thisYear[~thisYear.isin(prevYear) & thisYear.isin(long_time_ago)].nunique()
        sink = prevYear[~prevYear.isin(thisYear)].nunique()

        data_holder.append(
            pd.Series([i, y, new, trans, prev_old, sink], index=["cate", "year", "新", "續", "回", "流失"])
            )

data_holder = pd.DataFrame(data_holder)

In [18]:
y = 2020
i = "Car"
r = ""

thisYear = used_data.query("""
            year==@y & \
            cate==@i
            """).groupby(["id", "iroute"])["ipolicy"].count()
            
prevYear = used_data.query("""
                        year==@y-1 & \
                        cate==@i
                        """).groupby(["id", "iroute"])["ipolicy"].count()


In [21]:
def long_time_ago_byPerson(subset):
    tem = subset.groupby(["year", "iroute"])["ipolicy"].count()
    return tem.loc[subset["year"].max()]


long_time_ago = used_data.query("""
            year<@y-1 & \
            cate==@i
            """).groupby("id")\
                .apply(long_time_ago_byPerson)

In [96]:
data_holder
t = used_data.groupby("cate")["id"].nunique()
t = pd.concat([
    data_holder,
    pd.DataFrame(t).assign(year="不分年").reset_index().rename(columns={"id":"總人數"})
    ])
t.sort_values(["cate", "year"], inplace=True)
t["cate"] = t["cate"].apply(lambda x:
    {"Car": "車險", "F_PS":"住火", "H_PS":"個傷", "H_TV":"旅平", "H_GP":"團傷"}.get(x, x)
)


In [97]:
t.to_csv('../../data/人數_byYear_Cate.csv', index=False)

驗證

In [90]:
ply2020 = pd.read_csv(r'D:\新安\A&H\Car\data\2020\car_policy\policy.txt', dtype=str, sep='|')
irl = pd.read_csv(r'D:\新安\A&H\Car\data\2020\car_policy\ply_relation.txt', dtype=str, sep='|')

In [91]:
t = ply2020.merge(irl[['ipolicy', 'ilast_ply']], on='ipolicy', how='left')

In [95]:
print("保單續保率:", 1-irl["ilast_ply"].isna().mean())
print("人員續保率:", t[~t["ilast_ply"].isna()]["iassured"].nunique() / t["iassured"].nunique())


保單續保率: 0.6775670895345896
人員續保率: 0.7244585507240258


In [89]:
ply2020.query("ipolicy=='00009V7019469'")

,ipolicy,fassured,iassured,iassured_ext,ibirth,fsex,fmarriage,aassured_zip,iply_area,nbrand_abbr,...,iins_kit,mcover_limit,iassured_cntry,iapplicant_cntry,foccup,noccup,applicant_foccup,applicant_noccup,punknown_acc,nmile_ubi


In [73]:
raw_2020.head(2)

,ipolicy,fcard_class,irelative_ply,ilast_ply,inext_ply,itmp_policy,dply_begin,dply_end,ipro_year,ibrand,...,ibranch_at,fcomp_class,fcomp_class_last,qnext_year,irate_type,bzfrom,iroute_comm,qprovision,qdrunk_driving,isign
0,00009V7019469,2,NaN,NaN,NaN,00009CG020440,2020-09-22,2020-09-23,NaN,NaN,...,00,NaN,NaN,0,E,41,G02,0,0,882158
1,00009V7011419,2,NaN,NaN,NaN,00009CG011896,2020-07-19,2020-07-20,NaN,NaN,...,00,NaN,NaN,0,E,41,G02,0,0,882158
